# H A $\alpha$ decomposition evaluation

In this notebook we compare results of the new implementation of the
H A $\alpha$ decomposition with the C legacy version of PolSARpro.  
It assumes the user has a working installation of PolSARpro and has followed the 
instructions in the README.md file.

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
from polsarpro.util import read_PSP_bin, read_T3
from polsarpro.decompositions import h_a_alpha, h_a_alpha_dask
from pathlib import Path
import os
import matplotlib.pyplot as plt
import numpy as np
from spectral.io.envi import save_image

# change to your local C-PolSARpro install dir
c_psp_dir = "/home/c_psp/Soft/bin/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_process_sngl/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_convert/"

# change to your data paths
input_test_dir = Path("/data/psp/input/h_a_alpha_decomposition/")
# output_test_dir = Path("/data/res/freeman_cpsp")
output_test_dir = Path("/data/res/h_a_alpha_cpsp")
# output_test_dir = Path("/data/psp/output/freeman_decomposition/out")

## Run the C-version on some test data

In [ ]:
from polsarpro.devtools import parse_psp_parameter_string
import os
# fl1: alpha, beta, delta, lambda
# fl2: lambda
# fl3: alpha
# fl4: entropy 
# fl5: anisotropy
# fl6-9 combinations

input_str= f"""id: {input_test_dir} 
od: {output_test_dir}
iodf: T3
nwr: 7
nwc: 7
ofr: 0
ofc: 0
fnr: 18432
fnc: 1248
fl1: 1
fl2: 1
fl3: 1
fl4: 1
fl5: 1
fl6: 0
fl7: 0
fl8: 0
fl9: 0
errf: /tmp/MemoryAllocError.txt
mask: {input_test_dir}/mask_valid_pixels.bin
"""
result = parse_psp_parameter_string(input_str)
os.system(f"h_a_alpha_decomposition.exe {result}")

## Load coherency matrices and C-PSP outputs

In [ ]:
T3 = read_T3(input_test_dir)

out_names = ["entropy", "anisotropy", "alpha", "beta", "delta", "gamma", "lambda"]
out_c = {}

for name in out_names:
    file_name = output_test_dir / f"{name}.bin"
    out_c[name] = read_PSP_bin(file_name)

In [ ]:
flags = ("entropy", "anisotropy", "alpha", "beta", "delta", "gamma", "lambda")
out_p = h_a_alpha(T3, "T3", boxcar_size=[7, 7], flags=flags)

In [ ]:
for var in out_names:
    var_c = out_c[var]
    var_p = out_p[var]
    relerr = np.nanmean(((np.abs(var_p - var_c) / np.abs(var_c))))
    print(f"Relative error for {var}: {relerr}")

## Numeric and visual evaluation

In [ ]:
var_name = "H"
var_p = h
var_c = h_c

plt.figure(figsize=(10, 10))
plt.imshow(var_p[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ python")

plt.figure(figsize=(10, 10))
plt.imshow(var_c[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ C")

np.nanmean(((np.abs(var_p - var_c) / np.abs(var_c))))

In [ ]:
var_name = "A"
var_p = a
var_c = a_c

plt.figure(figsize=(10, 10))
plt.imshow(var_p[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ python")

plt.figure(figsize=(10, 10))
plt.imshow(var_c[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ C")

np.nanmean(((np.abs(var_p - var_c) / np.abs(var_c))))

In [ ]:
var_name = "alpha"
var_p = alpha
var_c = alpha_c

plt.figure(figsize=(10, 10))
plt.imshow(var_p[::8][1000:1500], interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ python")

plt.figure(figsize=(10, 10))
plt.imshow(var_c[::8][1000:1500], interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ C")

np.nanmean(((np.abs(var_p - var_c) / np.abs(var_c))))

## Apply the numpy implementation (single-thread, only for reference)

## Apply the dask implementation (multi-thread)

In [ ]:
h, a, alpha = h_a_alpha_dask(T3, "T3", boxcar_size=[7, 7])

# # write outputs
# save_image(hdr_file=output_test_dir / "Freeman_Odd_py.hdr", image=ps_da, force=True)
# save_image(hdr_file=output_test_dir / "Freeman_Dbl_py.hdr", image=pd_da, force=True)
# save_image(hdr_file=output_test_dir / "Freeman_Vol_py.hdr", image=pv_da, force=True)

In [ ]:
# print(f"difference between dask and normal implementation for odd {np.abs(ps - ps_da).sum()}, double {np.abs(pd - pd_da).sum()}, and volume {np.abs(pv - pv_da).sum()}")